# OPTaaS Scikit-learn Pipelines

Using the OPTaaS Python Client, you can optimize any scikit-learn pipeline. For each step or estimator in the pipeline, OPTaaS just needs to know what parameters to optimize and what constraints will apply to them.

Your pipeline can even include **optional** steps (such as feature selection), and **choice** steps (such as choosing between a set of classifiers).

We have provided pre-defined parameters and constraints for some of the most widely used estimators, such as Random Forest and XGBoost. The example below demonstrates how to use them. See also our [tutorial on defining your own custom optimizable estimators](sklearn_custom.ipynb).

## Load your dataset

We will run a classification pipeline using the German Credit Data available [here](https://newonlinecourses.science.psu.edu/stat857/node/215/). The data contains 1000 rows, with 20 feature columns and 1 target column which includes 2 classes.

In [1]:
import pandas as pd

data = pd.read_csv('german_credit.csv')
features = data[data.columns.drop(['Creditability'])]
target = data['Creditability']

## Specify your estimators

Our pipeline will include:
* An optional feature selection step using PCA
* A choice of classifier from: Random Forest, Extra Trees and Linear SVC

In [2]:
from mindfoundry.optaas.client.sklearn_pipelines.pca import PCA
from mindfoundry.optaas.client.sklearn_pipelines.forest import RandomForestClassifier, ExtraTreesClassifier
from mindfoundry.optaas.client.sklearn_pipelines.linear_svc import LinearSVC
from mindfoundry.optaas.client.sklearn_pipelines.mixin import choice, optional_step

estimators = [
    ('feature_selection', optional_step(PCA())),
    ('classification', choice(
        RandomForestClassifier(),
        ExtraTreesClassifier(),
        LinearSVC()
    ))
]

## Connect to the OPTaaS server using your API Key

We now create a client, and connect to the web service that will perform our optimization. You will need to input your personal API key. Make sure you keep your key private and don't commit it to your version control system. 

In [3]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Create your Sklearn Task

We don't need to worry about specifying all the parameters and constraints - they are generated based on the estimators and any additional parameters we specify. In this case we specify the `feature_count` and we set `gpu_enabled` to False.

In [4]:
task = client.create_sklearn_task(
    title='My Sklearn Task', 
    estimators=estimators,
    feature_count=len(features.columns)
)

display(task.parameters)
display(task.constraints)

[{'id': 'feature_selection',
  'items': [{'default': 'auto',
    'enum': ['arpack', 'auto', 'full', 'randomized'],
    'id': 'feature_selection__svd_solver',
    'name': 'svd_solver',
    'type': 'categorical'},
   {'choices': [{'id': 'feature_selection__n_components_int',
      'maximum': 20,
      'minimum': 1,
      'name': 'n_components_int',
      'type': 'integer'},
     {'id': 'feature_selection__n_components_float',
      'maximum': 0.9999999999999999,
      'minimum': 5e-324,
      'name': 'n_components_float',
      'type': 'number'},
     {'id': 'feature_selection__n_components_mle',
      'name': 'n_components_mle',
      'type': 'constant',
      'value': 'mle'}],
    'id': 'feature_selection__n_components',
    'includeInDefault': False,
    'name': 'n_components',
    'optional': True,
    'type': 'choice'},
   {'default': False,
    'id': 'feature_selection__whiten',
    'name': 'whiten',
    'type': 'boolean'},
   {'default': 0.0,
    'id': 'feature_selection__tol',
  

["if #feature_selection__svd_solver == 'arpack' then ( #feature_selection__n_components_int < 20 ) && #feature_selection__tol is_present",
 "if ( #feature_selection__svd_solver == 'auto' ) || ( #feature_selection__svd_solver == 'randomized' ) then #feature_selection__n_components is_absent || ( #feature_selection__n_components == #feature_selection__n_components_int )",
 'if #classification__0__bootstrap == false then #classification__0__oob_score != true',
 'if #classification__1__bootstrap == false then #classification__1__oob_score != true',
 "if #classification__2__dual == true then #classification__2__penalty == 'l2'",
 "if #classification__2__dual == false then #classification__2__loss == 'squared_hinge'",
 "if #classification__2__multi_class == 'ovr' then ( #classification__2__dual is_present && #classification__2__penalty is_present ) && #classification__2__loss is_present",
 "if #classification__2__multi_class == 'crammer_singer' then ( #classification__2__dual is_absent && #c

## Generate the first pipeline

We get the first configuration from OPTaaS and use it to generate a pipeline:

In [5]:
configuration = task.generate_configuration()
pipeline = task.make_pipeline(configuration)

display(pipeline)

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_l...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

## Run your pipeline and calculate the result

We define a function to run our pipeline and calculate the mean score:

In [6]:
from sklearn.model_selection import cross_val_score

def get_result(pipeline):
    scores = cross_val_score(pipeline, features, target, scoring='f1_micro')
    mean_score = scores.mean()
    return mean_score

mean_score = get_result(pipeline)

display(f'Mean score: {mean_score:.3f}')

'Mean score: 0.701'

## Record the result in OPTaaS and generate the next pipeline

In [7]:
configuration = task.record_result(configuration, score=mean_score)
pipeline = task.make_pipeline(configuration)

## Repeat as necessary

In [8]:
number_of_iterations = 20

for _ in range(number_of_iterations):
    display("OPTaaS recommendation:", pipeline)
    mean_score = get_result(pipeline)
    display(f'Mean score: {mean_score:.3f}')
    configuration = task.record_result(configuration, score=mean_score)
    pipeline = task.make_pipeline(configuration)

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.2962727774156646,
            min_impurity_split=None, min_samples_leaf=2,
            min_samples_split=16,
            min_weight_fraction_leaf=0.4143818575882208, n_estimators=58,
            n_jobs=1, oob_score=True, random_state=None, verbose=0,
            warm_start=False))])

'Mean score: 0.700'

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('classification', ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features=0.15095756071888777,
           max_leaf_nodes=None, min_impurity_decrease=0.5486902990992869,
           min_impurity_split=None, min_samples_leaf=9,
           min_samples_split=4,
           min_weight_fraction_leaf=0.1504150440690975, n_estimators=26,
           n_jobs=1, oob_score=True, random_state=None, verbose=0,
           warm_start=False))])

...

Pipeline(memory=None,
     steps=[('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=0.3337076920934011,
            max_leaf_nodes=None, min_impurity_decrease=0.6758142803303094,
            min_impurity_split=None, min_samples_leaf=2,
  ...            n_jobs=1, oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

'Mean score: 0.700'

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power=99, n_components=None, random_state=None,
  svd_solver='randomized', tol=None, whiten=True)), ('classification', ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=97, max_features=0.98090061191...6,
           n_jobs=1, oob_score=True, random_state=None, verbose=0,
           warm_start=False))])

'Mean score: 0.700'

## Complete your task

In [9]:
task.complete()

best_result, best_configuration = task.get_best_result_and_configuration()
best_pipeline = task.make_pipeline(best_configuration)

display(best_result, best_pipeline)

{ 'configuration': 'a13e0d03-019a-470f-b76a-1e87eb59ab16',
  'id': 486,
  'score': 0.723025420630211,
  'user_defined_data': None}

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.052710372024094065, whiten=True)), ('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_featur...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])